In [1]:
import pandas as pd
import numpy as np

import os

In [2]:
TRANSFORM_FOLDER = '../../DATA/TRANSFORM/entity_details/'
FINAL_FOLDER = '../../DATA/FINAL'

In [3]:
financial_ratios = pd.read_csv('../../DATA/MODEL/financial_ratios_df.csv')

C:\Users\matus.rohal\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3433: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
financial_ratios

,financial_report_id,financial_statement_id,financial_report_order_number,entity_id,approved_date,entity_ico,year,month,financial_report_template_id,entity_name,...,CF_SAL,ROS,EAT_YIE,ROI,ROA_NETTO,LABOR_PRODUCTIVITY,SOCLIAB_SAL,sknace_division_name,sknace_division,sknace_subcategory
0,4890214,2575566,2,25527,2015-07-31,671.0,2014,1,699.0,"Kerametal, akciová spoločnosť,",...,-86.265512,-86.618740,-49.939078,-0.106281,-0.106281,0.217953,6.666929,"Veľkoobchod, okrem motorových vozidiel a motoc...",G,46.0
1,5392251,2976863,2,25527,2016-12-31,671.0,2015,1,699.0,"Kerametal, akciová spoločnosť, Bratislava",...,-89.959262,-90.091385,-32.236974,-0.063249,-0.063249,0.000000,11.652904,"Veľkoobchod, okrem motorových vozidiel a motoc...",G,46.0
2,5887438,3375702,2,25527,2017-12-27,671.0,2016,1,699.0,"Kerametal, akciová spoločnosť, Bratislava",...,-2589.352056,-2589.592778,-1478.707904,-346.530300,-346.530300,0.000000,21.230692,"Veľkoobchod, okrem motorových vozidiel a motoc...",G,46.0
3,6361460,3757740,1,25527,2018-12-17,671.0,2017,1,699.0,"Kerametal, akciová spoločnosť, Bratislava",...,-15.467637,-15.708480,-11.531123,-1.320779,-2.124525,0.000000,21.241846,"Veľkoobchod, okrem motorových vozidiel a motoc...",G,46.0
4,6417420,3787505,1,25527,NaN,671.0,2018,1,699.0,"Kerametal, akciová spoločnosť, Bratislava",...,-1175.036293,-1175.036293,-139.693428,-204.590499,-204.590499,0.000000,27.436811,"Veľkoobchod, okrem motorových vozidiel a motoc...",G,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679945,8105494,5156278,1,2040228,2022-05-31,54329841.0,2021,1,699.0,skdarceky s. r. o.,...,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,"Veľkoobchod, okrem motorových vozidiel a motoc...",G,46.0
1679946,7514307,4679250,1,1957322,2021-03-20,56488041.0,2020,12,687.0,"NOKO Land, s.r.o.",...,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,"Administratívne, pomocné kancelárske a iné obc...",N,82.0
1679947,4458450,2225050,1,289353,2015-02-27,60748940.0,2014,1,699.0,VALPEX s.r.o.,...,-0.004337,-0.009830,-0.009618,-0.024555,-0.024584,0.095681,0.006934,"Veľkoobchod, okrem motorových vozidiel a motoc...",G,46.0
1679948,5030426,2678436,1,289353,2016-03-12,60748940.0,2015,1,699.0,VALPEX s.r.o.,...,-0.026090,-0.034069,-0.033136,-0.136401,-0.136501,0.074570,0.000000,"Veľkoobchod, okrem motorových vozidiel a motoc...",G,46.0


## Joining financial issues indicators

In [5]:
# pomer VI a zavazkov je mensi ako 8, t.j EQ_TL < 0.08
# 2016 : 0.04, 2017 : 0.06, 2018 : 0.08, 
# EQ < 0
# L3 < 1

In [6]:
def process_issues(row):
    EQ_TL = row.EQ_TL
    EQ = row.EQ
    L2 = row.L2
    EAT = row.EAT
    y = row.year

    if y <= 2016: threat_criterion = 0.04
    elif y == 2017: threat_criterion = 0.06
    else: threat_criterion = 0.08

    if EQ_TL < threat_criterion and EQ < 0 and L2 < 1 and EAT < 0:
        return 1
    return 0
    

In [7]:
legal_financial_issues_criterion = []
for i, row in financial_ratios.iterrows():
    legal_financial_issues_criterion.append(
        (row.entity_ico,
        row.year,
        process_issues(row))      
    )

financial_issues_criterion_df = pd.DataFrame(legal_financial_issues_criterion, columns=['entity_ico', 'year', 'RUZ_indirect_criterion'])
display(financial_issues_criterion_df)

,entity_ico,year,RUZ_indirect_criterion
0,671.0,2014,0
1,671.0,2015,0
2,671.0,2016,1
3,671.0,2017,1
4,671.0,2018,1
...,...,...,...
1679945,54329841.0,2021,0
1679946,56488041.0,2020,0
1679947,60748940.0,2014,0
1679948,60748940.0,2015,0


In [8]:
financial_issues_criterion_info = financial_issues_criterion_df.query('RUZ_indirect_criterion > 0').drop(columns='RUZ_indirect_criterion')
financial_issues_criterion_info['event_type'] = 'RUZ_indirect_criterion'

In [9]:
files = [
 'insolvency_register.csv',
 'vat_registration_cancellation.csv']

In [10]:
data = dict()
data['insolvency_register'] = pd.read_csv(TRANSFORM_FOLDER + 'insolvency_register.csv')
data['vat_registration_cancellation'] = pd.read_csv(TRANSFORM_FOLDER + 'vat_registration_cancellation.csv')

In [11]:
insolvency_info = data['insolvency_register'].copy()
insolvency_info['proceeding_type'] = insolvency_info['proceeding_type'].apply(lambda x: 'ostatne_konania' if x not in ['konkurz', 'restrukturalizacia '] else x)
insolvency_info

insolvency_info.drop_duplicates(['entity_ico', 'proceeding_type', 'proceeding_month'], inplace=True)

insolvency_info = insolvency_info.sort_values('proceeding_month', ascending=False).groupby(['entity_ico', 'proceeding_type', 'proceeding_month']).tail(1)

insolvency_info.rename(columns={'proceeding_type': 'event_type', 'proceeding_month': 'year'}, inplace=True)

insolvency_info = insolvency_info[['entity_ico', 'year', 'event_type']]

insolvency_info['event_type'] = insolvency_info['event_type'].apply(lambda x: 'RU_' + x)
insolvency_info


,entity_ico,year,event_type
0,53051505.0,2023,RU_konkurz
276,45380058.0,2023,RU_konkurz
287,45257337.0,2023,RU_konkurz
286,35964065.0,2023,RU_konkurz
285,44353260.0,2023,RU_konkurz
...,...,...,...
7033,31105424.0,1993,RU_ostatne_konania
7032,31655785.0,1993,RU_ostatne_konania
6995,598321.0,1993,RU_ostatne_konania
7019,31412874.0,1993,RU_ostatne_konania


In [12]:
vat_registration_cancellation_info = data['vat_registration_cancellation'].copy()
# vat_registration_cancellation_info

vat_registration_cancellation_info['entity_ico'] = pd.to_numeric(vat_registration_cancellation_info['entity_ico'], errors = 'coerce')
vat_registration_cancellation_info.query('entity_ico == entity_ico', inplace=True)
vat_registration_cancellation_info = vat_registration_cancellation_info[['entity_ico', 'vat_cancellation_year']].drop_duplicates(['entity_ico'])
vat_registration_cancellation_info['vat_cancellation_year'] = pd.to_numeric(vat_registration_cancellation_info['vat_cancellation_year'])
vat_registration_cancellation_info['event_type'] = 'DPHZ_vat_registration_cancelled'
vat_registration_cancellation_info.rename(columns = {'vat_cancellation_year': 'year'}, inplace=True)
vat_registration_cancellation_info

,entity_ico,year,event_type
0,37712080.0,2020,DPHZ_vat_registration_cancelled
1,37257455.0,2018,DPHZ_vat_registration_cancelled
2,37713256.0,2013,DPHZ_vat_registration_cancelled
3,35125730.0,2020,DPHZ_vat_registration_cancelled
4,37338048.0,2020,DPHZ_vat_registration_cancelled
...,...,...,...
4749,2083477.0,2018,DPHZ_vat_registration_cancelled
4750,50635808.0,2023,DPHZ_vat_registration_cancelled
4751,6199917.0,2020,DPHZ_vat_registration_cancelled
4752,53714521.0,2023,DPHZ_vat_registration_cancelled


In [13]:
# ru_entity_details = data['entity_details'].copy()
# ru_entity_details.query('financial_statements_list == financial_statements_list', inplace= True)

# ru_entity_details = (ru_entity_details
#                      .merge(insolvency_info, on = 'entity_ico', how = 'left')
#                      .merge(vat_registration_cancellation_info, on = 'entity_ico', how = 'left'))


# ru_entity_details['establishment_date'] = pd.to_datetime(ru_entity_details['establishment_date'], errors='coerce')
# ru_entity_details['cancellation_date'] = pd.to_datetime(ru_entity_details['cancellation_date'], errors='coerce')
# ru_entity_details['cancellation_date_RO'] = pd.to_datetime(ru_entity_details['cancellation_date_RO'], errors='coerce')

# ru_entity_details = ru_entity_details.query('establishment_date == establishment_date and id not in [1019370, 1063773, 1068320]')

# ru_entity_details = ru_entity_details[['entity_ico', 'establishment_date', 'cancellation_date', 'cancellation_date_RO']]

# ru_entity_details['establishment_year'] = ru_entity_details['establishment_date'].apply(lambda x: x.year if x==x else np.nan)
# ru_entity_details['cancellation_year'] = ru_entity_details['cancellation_date'].apply(lambda x: x.year if x==x else np.nan)
# ru_entity_details['cancellation_year_RO'] = ru_entity_details['cancellation_date_RO'].apply(lambda x: x.year if x==x else np.nan)

# ru_entity_details = ru_entity_details[['entity_ico', 'establishment_year', 'cancellation_year', 'cancellation_year_RO']]
# ru_entity_details.columns = ['entity_ico', 'RUZ_established', 'RUZ_cancelled', 'RO_cancelled']
# ru_entity_events = ru_entity_details.melt(id_vars='entity_ico')
# ru_entity_events.columns = ['entity_ico', 'event_type', 'year']
# ru_entity_events.query('year == year', inplace = True)
# ru_entity_events['year'] = ru_entity_events['year'].apply(lambda x: x if isinstance(x, float) else x.year)
# ru_entity_events

In [14]:
valid_ico_list = list(financial_ratios['entity_ico'].unique())

In [15]:
entity_issues_events_long = (pd.concat(
    [
         insolvency_info,
        #  ru_entity_events,
         vat_registration_cancellation_info,
         financial_issues_criterion_info
    ], axis = 0))

entity_issues_events_long['event_type'].value_counts()

RUZ_indirect_criterion             207351
RU_konkurz                           5763
DPHZ_vat_registration_cancelled      4753
RU_ostatne_konania                   1269
Name: event_type, dtype: int64

In [16]:
entity_issues_events_long.query('event_type not in ["RUZ_established", "RO_cancelled"] and entity_ico in @valid_ico_list and year >= 2014', inplace=True)

entity_issues_events_long['event_type'].value_counts()

RUZ_indirect_criterion             207351
DPHZ_vat_registration_cancelled      3608
RU_konkurz                           3055
RU_ostatne_konania                    391
Name: event_type, dtype: int64

In [17]:
entity_issues_events_long['indicator'] = 1
entity_issues_events_long.to_csv('../../DATA/TRANSFORM/issues_events.csv', index = False)

In [18]:

entity_issues_indicators_1y_off = entity_issues_events_long.copy()
entity_issues_indicators_2y_off = entity_issues_events_long.copy()

entity_issues_indicators_1y_off['year'] = entity_issues_indicators_1y_off['year'] - 1
entity_issues_indicators_1y_off['event_type'] = entity_issues_indicators_1y_off['event_type'].apply(lambda x: x+'_1y_off')

entity_issues_indicators_2y_off['year'] = entity_issues_indicators_2y_off['year'] - 2
entity_issues_indicators_2y_off['event_type'] = entity_issues_indicators_2y_off['event_type'].apply(lambda x: x+'_2y_off')

entity_issues_indicators = (pd.concat(
    [
        entity_issues_events_long.copy(),
        entity_issues_indicators_1y_off,
        entity_issues_indicators_2y_off
    ], axis=0
))

entity_issues_events = pd.pivot_table(entity_issues_indicators, index = ['entity_ico', 'year'], columns = 'event_type', values='indicator', fill_value=0)
entity_issues_events.reset_index(inplace=True)
entity_issues_events

event_type,entity_ico,year,DPHZ_vat_registration_cancelled,DPHZ_vat_registration_cancelled_1y_off,DPHZ_vat_registration_cancelled_2y_off,RUZ_indirect_criterion,RUZ_indirect_criterion_1y_off,RUZ_indirect_criterion_2y_off,RU_konkurz,RU_konkurz_1y_off,RU_konkurz_2y_off,RU_ostatne_konania,RU_ostatne_konania_1y_off,RU_ostatne_konania_2y_off
0,671.0,2014,0,0,0,0,0,1,0,0,0,0,0,0
1,671.0,2015,0,0,0,0,1,1,0,0,0,0,0,0
2,671.0,2016,0,0,0,1,1,1,0,0,1,0,0,0
3,671.0,2017,0,0,0,1,1,0,0,1,0,0,0,0
4,671.0,2018,0,0,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416803,54300681.0,2022,0,1,0,0,0,0,0,0,0,0,0,0
416804,54300681.0,2023,1,0,0,0,0,0,0,0,0,0,0,0
416805,54303575.0,2021,0,0,1,0,0,0,0,0,0,0,0,0
416806,54303575.0,2022,0,1,0,0,0,0,0,0,0,0,0,0


In [19]:
ISSUES_TYPES = ['DPHZ_vat_registration_cancelled', 'RUZ_indirect_criterion', 'RU_konkurz', 'RU_ostatne_konania']

In [20]:
def process_entity_issues(row, issue_type):
        return max(row[issue_type+'_1y_off'], row[issue_type+'_2y_off'])

for issue_type in ISSUES_TYPES:
    entity_issues_events[issue_type+'_2y_off'] = entity_issues_events.apply(lambda x: process_entity_issues(x, issue_type), axis = 1)

entity_issues_events

event_type,entity_ico,year,DPHZ_vat_registration_cancelled,DPHZ_vat_registration_cancelled_1y_off,DPHZ_vat_registration_cancelled_2y_off,RUZ_indirect_criterion,RUZ_indirect_criterion_1y_off,RUZ_indirect_criterion_2y_off,RU_konkurz,RU_konkurz_1y_off,RU_konkurz_2y_off,RU_ostatne_konania,RU_ostatne_konania_1y_off,RU_ostatne_konania_2y_off
0,671.0,2014,0,0,0.0,0,0,1.0,0,0,0.0,0,0,0.0
1,671.0,2015,0,0,0.0,0,1,1.0,0,0,0.0,0,0,0.0
2,671.0,2016,0,0,0.0,1,1,1.0,0,0,1.0,0,0,0.0
3,671.0,2017,0,0,0.0,1,1,1.0,0,1,1.0,0,0,0.0
4,671.0,2018,0,0,0.0,1,0,0.0,1,0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416803,54300681.0,2022,0,1,1.0,0,0,0.0,0,0,0.0,0,0,0.0
416804,54300681.0,2023,1,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0
416805,54303575.0,2021,0,0,1.0,0,0,0.0,0,0,0.0,0,0,0.0
416806,54303575.0,2022,0,1,1.0,0,0,0.0,0,0,0.0,0,0,0.0


In [21]:
display(financial_ratios.shape[0])

financial_ratios_copy = financial_ratios.merge(entity_issues_events, on = ['entity_ico', 'year'], how = 'left')
display(financial_ratios.shape[0])


1679950

1679950

In [22]:
financial_ratios_copy['cancellation_year'] = pd.to_datetime(financial_ratios_copy['cancellation_date'], errors='coerce').dt.year

In [23]:
def ruz_cancelled(row):
    if row.year >= row.cancellation_year: return 1
    return 0

def ruz_cancelled_1y_off(row):
    if row.year + 1 >= row.cancellation_year: return 1
    return 0

def ruz_cancelled_2y_off(row):
    if row.year + 2 >= row.cancellation_year: return 1
    return 0


financial_ratios_copy['RUZ_cancelled'] = financial_ratios_copy.apply(ruz_cancelled, axis=1)
financial_ratios_copy['RUZ_cancelled_1y_off'] = financial_ratios_copy.apply(ruz_cancelled_1y_off, axis=1)
financial_ratios_copy['RUZ_cancelled_2y_off'] = financial_ratios_copy.apply(ruz_cancelled_2y_off, axis=1)

In [24]:
for col in ['DPHZ_vat_registration_cancelled',
 'DPHZ_vat_registration_cancelled_1y_off',
 'DPHZ_vat_registration_cancelled_2y_off',
 'RUZ_indirect_criterion',
 'RUZ_indirect_criterion_1y_off',
 'RUZ_indirect_criterion_2y_off',
 'RU_konkurz',
 'RU_konkurz_1y_off',
 'RU_konkurz_2y_off',
 'RU_ostatne_konania',
 'RU_ostatne_konania_1y_off',
 'RU_ostatne_konania_2y_off',
 'RUZ_cancelled',
 'RUZ_cancelled_1y_off',
 'RUZ_cancelled_2y_off']:
    financial_ratios_copy[col] = financial_ratios_copy[col].fillna(0)



In [25]:
financial_ratios_copy.to_csv('../../DATA/MODEL/training_data_temp.csv', index = False)

In [26]:
entity_issues_indicators = entity_issues_indicators.query('entity_ico > 0')[['entity_ico', 'year'] + sorted(list(entity_issues_indicators.columns)[2:])].sort_values(['entity_ico', 'year'])

In [27]:
entity_issues_indicators.to_csv('../../DATA/TRANSFORM/issues_indicators.csv', index = False)